In [37]:
using Distributions
using Random
using LinearAlgebra

In [20]:
beta = [1, 2, 0,0,0,0,0,0,0]

9

In [34]:
# sample data creation
Random.seed!(123)
n = 50
std = 0.5
du = Uniform()
Xvar = rand(du, n, 8)
constant = ones(n)
X = hcat(constant, Xvar*2)

dn = Normal()
eps = rand(dn, n)

y = X*beta + std*eps;

In [41]:
# set prior
m = zeros(9)
C = zeros((9,9)) + I
a = 1.0
b = 1.0;

In [ ]:
# MCMC
ns = 1000
burn= 0.1*ns
betaresult = zeros((9, ns+1))
phiresult = zeros(ns+1)

# initialize
b0 = m
phi0 = a/b
phiresult[1] = phi0

# loop
# まずはMvNormalを使う
sigmainv = X'X + inv(C)
sigmainvt = sigmainv'
sigma = inv(sigmainv)

posa = n/2 + a

for i in 1:ns
    
    if i == 1
        mu = (phiresult[i]*X'y + m)\sigmainvt
    else
        mu = (phiresult[i-1]*X'y + m)\sigmainvt
    end
    
    cbeta = rand(MvNormal(mu, sigma))
    betaresult[:,i] = cbeta
    phiresult[i] = rand(Gamma(posa, (y-X*cbeta)'(y-X*cbeta)/2 + b))
end

    